In [ ]:
import numpy as np

In [ ]:
# 1: 计算保型分数，这里n = calib_Y.shape[0]，即校准数据集的样本数
cal_smx = model(calib_X).softmax(dim=1).numpy() #使用模型对校准数据集进行预测，计算softmax输出。输出是一个二维数组形状为(n, num_classes)，n为样本数，num_classes为类别数
cal_scores = 1-cal_smx[np.arange(n),cal_labels] #计算保型分数，即1-softmax输出的真实标签对应的概率。这里是提取每个样本对应的真实标签的概率，然后计算1-概率值，共n个样本
# 2: 获取调整后的分位数
q_level = np.ceil((n+1)*(1-alpha))/n #计算调整后的分位数水平q_level，这里alpha时显著性水平（如0.1表示90%置信），np.ceil时将输入的数值向上取整到最近的整数
qhat = np.quantile(cal_scores, q_level, method= 'higher') #计算保型分数（给定数据）的q_level分位数qhat。np.quantile用于根据指定的分位数水平返回数据中的相应值（对数据进行排序后，在特定比例处的值），cal_scores是一维数组（保型分数），q_level是分位数水平，method='higher'表示是计算分位数的方法，higher方法确保返回的分位数是大于或等于指定分位数水平的最小值
val_smx = model(val_X).softmax(dim=1).numpy() #使用模型对验证数据集进行预测，计算softmax输出
prediction_sets = val_smx >= (1-qhat) # 3: 形成预测集，对于每个样本，如果模型对某个类别的预测概率大于等于1-qhat，则该类别包含在预测集中

In [ ]:
#1.计算校准数据集的保型分数，这里n代表校准数据集的样本数量，calib_X.shape[0]==calib_Y.shape[0]==n
##这里cal_smx是模型对校准数据集的预测结果，是一个二维数组形状为(n, num_classes)，n为样本数，num_classes为类别数
cal_pi=cal_smx.argsort(1)[:,::-1] #这里首先对cal_smx进行排序然后反向排序（按列，相当于将每个样本的softmax类别概率从大到小排列）
cal_srt= np.take_along_axis(cal_smx,cal_pi,axis=1).cumsum(axis=1) #
cal_scores =np.take_along_axis(cal_srt,cal_pi.argsort(axis=1),axis=1)[range(n),cal_labels]
#Getthe scorequantile
qhat=np.quantile(cal_scores, np.ceil((n+1)*(1-alpha))/n,interpolation=higher)
#Deploy(output=listoflengthn,eachelementistensorofclasses)
val_pi=val_smx.argsort(1)[:,::-1];val_srt= np.take_along_axis(val_smx,val_pi,axis=1).cumsum(axis=1)
prediction_sets =np.take_along_axis(val_srt<=qhat,val_pi.argsort(axis=1),axis=1)

In [ ]:
# 1: 计算保型分数
#这里cal_labels是校准数据集的真实标签，cal_X是校准数据集的特征，model_upper和model_lower是模型的上界和下界，
cal_scores = np.maximum(cal_labels-model_upper(cal_X), model_lower(cal_X)-cal_labels)
# Get the score quantile
qhat = np.quantile(cal_scores, np.ceil((n+1)*(1-alpha))/n, interpolation= higher)
# Deploy (output=lower and upper adjusted quantiles)
prediction_sets = [val_lower- qhat, val_upper + qhat]